In [1]:
import os
os.chdir('..')

from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "knn7d"
fs_method, fs_ratio = "dnp", 50/142
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from utilities import GNNClassifier

model_fn = functools.partial(
    GNNClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GNNClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9093 | AUPR: 0.6018 | Acc: 94.98%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9159 | AUPR: 0.6161 | Acc: 95.07%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9190 | AUPR: 0.6282 | Acc: 95.24%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9115 | AUPR: 0.6065 | Acc: 94.97%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9169 | AUPR: 0.6201 | Acc: 95.12%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9185 | AUPR: 0.6293 | Acc: 95.21%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9190 | AUPR: 0.6282 | Acc: 95.24%
Internal test: AUC: 0.9232 | AUPR: 0.6720 | Acc: 95.11%
External test: AUC: 0.9161 | AUPR: 0.6141 | Acc: 94.79%


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9077 | AUPR: 0.5983 | Acc: 94.91%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9153 | AUPR: 0.6152 | Acc: 95.15%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9181 | AUPR: 0.6261 | Acc: 95.20%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9101 | AUPR: 0.5997 | Acc: 94.99%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9153 | AUPR: 0.6124 | Acc: 95.04%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9187 | AUPR: 0.6273 | Acc: 95.15%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9187 | AUPR: 0.6273 | Acc: 95.15%
Internal test: AUC: 0.9214 | AUPR: 0.6658 | Acc: 95.16%
External test: AUC: 0.9190 | AUPR: 0.6250 | Acc: 94.98%


In [4]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9146 | AUPR: 0.6115 | Acc: 95.06%
model(C=10.0)
Valid:         AUC: 0.9145 | AUPR: 0.6108 | Acc: 95.05%
model(C=100.0)
Valid:         AUC: 0.9145 | AUPR: 0.6104 | Acc: 95.04%
best model(C=1.0)
Valid:         AUC: 0.9146 | AUPR: 0.6115 | Acc: 95.06%
Internal test: AUC: 0.9171 | AUPR: 0.6542 | Acc: 94.96%
External test: AUC: 0.9133 | AUPR: 0.5958 | Acc: 94.65%


In [5]:
import warnings
from pytorch_tabnet.tab_model import TabNetClassifier
warnings.filterwarnings("ignore", category=UserWarning)

model_fn = functools.partial(
    TabNetClassifier, seed=SEED, 
    n_d=4, n_a=4, momentum=0.05, 
    n_steps=3, gamma=1.75, cat_emb_dim=1,
    n_independent=2, n_shared=2,
    verbose=0)
param_grid_list = [{
    'optimizer_params': [dict(lr=lr) for lr in [5e-2]],
    'gamma': [1.5, 1.75, 2.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)
warnings.resetwarnings()

TabNetClassifier
model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9036 | AUPR: 0.5847 | Acc: 94.96%
model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9046 | AUPR: 0.5921 | Acc: 94.78%
model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9011 | AUPR: 0.5898 | Acc: 94.37%
best model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9046 | AUPR: 0.5921 | Acc: 94.78%
Internal test: AUC: 0.9067 | AUPR: 0.6322 | Acc: 94.89%
External test: AUC: 0.9091 | AUPR: 0.5958 | Acc: 94.46%


In [6]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9033 | AUPR: 0.6085 | Acc: 86.03%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9041 | AUPR: 0.6133 | Acc: 86.07%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9046 | AUPR: 0.6147 | Acc: 85.91%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9025 | AUPR: 0.6072 | Acc: 85.65%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9040 | AUPR: 0.6127 | Acc: 85.78%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9045 | AUPR: 0.6130 | Acc: 85.68%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9025 | AUPR: 0.6072 | Acc: 85.65%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9040 | AUPR: 0.6127 | Acc: 85.78%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9045 | AUPR: 0.6130 | Acc: 85.68%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9046 | AUPR: 0.6147 | Acc: 8

In [7]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9126 | AUPR: 0.5703 | Acc: 94.51%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9145 | AUPR: 0.5799 | Acc: 94.56%
model(kernel=linear, C=1)
Valid:         AUC: 0.9143 | AUPR: 0.5801 | Acc: 94.55%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9145 | AUPR: 0.5799 | Acc: 94.56%
Internal test: AUC: 0.9167 | AUPR: 0.6347 | Acc: 94.62%
External test: AUC: 0.9150 | AUPR: 0.5706 | Acc: 94.34%


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_fn = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9161 | AUPR: 0.6119 | Acc: 95.05%
model(criterion=squared_error)
Valid:         AUC: 0.9161 | AUPR: 0.6118 | Acc: 95.05%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9161 | AUPR: 0.6119 | Acc: 95.05%
Internal test: AUC: 0.9159 | AUPR: 0.6460 | Acc: 94.96%
External test: AUC: 0.9162 | AUPR: 0.6074 | Acc: 94.76%


In [9]:
from sklearn.ensemble import AdaBoostClassifier
model_fn = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9116 | AUPR: 0.5902 | Acc: 94.90%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9130 | AUPR: 0.5929 | Acc: 94.91%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9128 | AUPR: 0.5955 | Acc: 94.94%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9130 | AUPR: 0.5929 | Acc: 94.91%
Internal test: AUC: 0.9130 | AUPR: 0.6388 | Acc: 94.72%
External test: AUC: 0.9117 | AUPR: 0.5803 | Acc: 94.48%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7654 | AUPR: 0.3851 | Acc: 94.02%
model(n_neighbors=5)
Valid:         AUC: 0.8005 | AUPR: 0.4390 | Acc: 94.44%
model(n_neighbors=7)
Valid:         AUC: 0.8155 | AUPR: 0.4620 | Acc: 94.54%
best model(n_neighbors=7)
Valid:         AUC: 0.8155 | AUPR: 0.4620 | Acc: 94.54%
Internal test: AUC: 0.8101 | AUPR: 0.4942 | Acc: 94.33%
External test: AUC: 0.8215 | AUPR: 0.4865 | Acc: 94.35%


In [11]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8834 | AUPR: 0.5042 | Acc: 81.06%
model(criterion=log_loss)
Valid:         AUC: 0.8767 | AUPR: 0.5014 | Acc: 81.38%
model(criterion=entropy)
Valid:         AUC: 0.8767 | AUPR: 0.5014 | Acc: 81.38%
best model(criterion=gini)
Valid:         AUC: 0.8834 | AUPR: 0.5042 | Acc: 81.06%
Internal test: AUC: 0.8696 | AUPR: 0.5313 | Acc: 74.70%
External test: AUC: 0.8681 | AUPR: 0.4890 | Acc: 74.58%


In [12]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8616 | AUPR: 0.3569 | Acc: 82.38%
best model()
Valid:         AUC: 0.8616 | AUPR: 0.3569 | Acc: 82.38%
Internal test: AUC: 0.8707 | AUPR: 0.4091 | Acc: 82.31%
External test: AUC: 0.8578 | AUPR: 0.3629 | Acc: 80.90%


In [13]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)